In [1]:
import pandas as pd
import requests
import base64
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import statistics as stat

In [2]:
from sqlalchemy import create_engine
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
engine = create_engine('sqlite:///pitchfork.sqlite')
conn = engine.connect()
review_data = pd.read_sql("SELECT * FROM reviews", conn)
artists_data = pd.read_sql("SELECT * FROM artists", conn)
content_data = pd.read_sql("SELECT * FROM content", conn)
genres_data = pd.read_sql("SELECT * FROM genres", conn)
labels_data = pd.read_sql("SELECT * FROM labels", conn)
years_data = pd.read_sql("SELECT * FROM years", conn)
master_data = pd.DataFrame(review_data.merge(genres_data, how = 'left')).merge(labels_data, how = 'left')

In [4]:
master_data.head()

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,genre,label
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017,electronic,virgin
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017,metal,hathenter
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,rock,static shock
3,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,rock,fashionable idiots
4,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017,rock,kill rock stars


In [5]:
master_data['title'][0]

'mezzanine'

In [6]:
numberofreviews = len(master_data['title'])
print(numberofreviews)

24909


In [7]:
client_id = 'YOUR CLIENT ID HERE'
client_secret = 'YOUR CLIENT SECRET HERE'

In [8]:
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [10]:
album_titles = []
album_artists = []
album_in_spotify = []
album_pop_score = []


for i in range(numberofreviews):
    album = master_data['title'][i]
    try:
        result = sp.search(album, type='album')
        album_id = result['albums']['items'][0]['id']
        album_name = result['albums']['items'][0]['name']
        artist = result['albums']['items'][0]['artists'][0]['name']
        number_tracks = result['albums']['items'][0]['total_tracks']
        album_titles.append(album_name)
        album_artists.append(artist)
        album_in_spotify.append("Yes")
        
    except:
        album_titles.append(album)
        album_artists.append("Not in Spotify")
        album_in_spotify.append("Not in Spotify")

    try:
        all_tracks_ids = []
        all_tracks = sp.album_tracks(album_id)
        n=0
        for n in range(number_tracks):
            all_tracks_ids.append(all_tracks['items'][n]['id'])
  
        all_tracks_popularity=[]
        all_tracks_results = sp.tracks(all_tracks_ids)
        y=0
        for y in range(number_tracks):
            all_tracks_popularity.append(all_tracks_results['tracks'][y]['popularity'])

        if sum(all_tracks_popularity)>=0:
            album_avg_popularity = stat.mean(all_tracks_popularity)
        else:
            album_avg_popularity = 0

        album_pop_score.append(album_avg_popularity)

    except:
        album_pop_score.append(0)

    print(str(i) + " " + album_name + ", " + artist + ", " + str(album_avg_popularity))


0 Mezzanine, Massive Attack, 53.72727272727273
1 Prelapsarian, Krallice, 7
2 All Of Them Naturals, Uranium Club, 27
3 All Of Them Naturals, Uranium Club, 27
4 Songs People Actually Liked - Volume 1 - The First 10 Years (1994-2003), Bowling For Soup, 29.166666666666668
5 Songs People Actually Liked - Volume 1 - The First 10 Years (1994-2003), Bowling For Soup, 29.166666666666668
6 Start of Something New, Melodyia Music, 59
7 Insecure: Music from the HBO Original Series, Season 2, Various Artists, 26.08695652173913
8 Stillness In Wonderland (Deluxe Edition), Little Simz, 35.21739130434783
9 Steve Reich: Tehillim / The Desert Music, Steve Reich, 12.181818181818182
10 Reflection (Deluxe), Fifth Harmony, 55
11 Reflection (Deluxe), Fifth Harmony, 55
12 Clear Sounds/Perfetta, Harry Bertoia, 0.5
13 Run the Jewels 3, Run The Jewels, 50.5
14 Nadir, Black Tongue, 32.8
15 Nadir, Black Tongue, 32.8
16 Nadir, Black Tongue, 32.8
17 Don't Smoke Rock, Smoke Dza, 29.615384615384617
18 Don't Smoke Rock, 

In [11]:
print(len(album_titles))
print(len(album_artists))
print(len(album_in_spotify))
print(len(album_pop_score))

24909
24909
24909
24909


In [56]:
dict = {"title_spotify":album_titles, "artist_spotify": album_artists, "Found_in_Spotify": album_in_spotify, "popularity":album_pop_score}

In [57]:
spotipy_df = pd.DataFrame(dict)
spotipy_df['title_lower'] = spotipy_df['title_spotify'].str.lower()
spotipy_df['artist_lower'] = spotipy_df['artist_spotify'].str.lower()

In [58]:
spotipy_df.head()

,title_spotify,artist_spotify,Found_in_Spotify,popularity,title_lower,artist_lower
0,Mezzanine,Massive Attack,Yes,53.727273,mezzanine,massive attack
1,Prelapsarian,Krallice,Yes,7.000000,prelapsarian,krallice
2,All Of Them Naturals,Uranium Club,Yes,27.000000,all of them naturals,uranium club
3,All Of Them Naturals,Uranium Club,Yes,27.000000,all of them naturals,uranium club
4,Songs People Actually Liked - Volume 1 - The F...,Bowling For Soup,Yes,29.166667,songs people actually liked - volume 1 - the f...,bowling for soup


In [59]:
master_data.head()

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,genre,label
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017,electronic,virgin
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017,metal,hathenter
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,rock,static shock
3,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,rock,fashionable idiots
4,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017,rock,kill rock stars


In [60]:
combined_df = master_data.merge(spotipy_df, left_on=["title", "artist"], right_on=["title_lower", "artist_lower"] , how="left" )

combined_df.head()

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,...,pub_month,pub_year,genre,label,title_spotify,artist_spotify,Found_in_Spotify,popularity,title_lower,artist_lower
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,...,1,2017,electronic,virgin,Mezzanine,Massive Attack,Yes,53.727273,mezzanine,massive attack
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,...,1,2017,metal,hathenter,Prelapsarian,Krallice,Yes,7.000000,prelapsarian,krallice
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,...,1,2017,rock,static shock,All Of Them Naturals,Uranium Club,Yes,27.000000,all of them naturals,uranium club
3,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,...,1,2017,rock,static shock,All Of Them Naturals,Uranium Club,Yes,27.000000,all of them naturals,uranium club
4,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,...,1,2017,rock,fashionable idiots,All Of Them Naturals,Uranium Club,Yes,27.000000,all of them naturals,uranium club


In [1]:
combined_df["title"].count()

NameError: name 'combined_df' is not defined

In [63]:
combined_df.to_csv('Merged_Data.csv')